In [1]:
import math
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import re
newd=[]
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec


In [2]:
from nltk.corpus import stopwords
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

import praw
from requests import Session
reddit = praw.Reddit(
    client_id="uRKvUgH0mRJqx3KWrni1AA",
    client_secret="MdP0rgI-cGYOwCOIWyBYvO5F0sIZnQ",
    password="CEna14!@",
    user_agent="testscript by u/fakebot3",
    username="Zealousideal_Tie15", check_for_async=True
)
infl =["Elon Musk", "Taylor Swift","Donald Trump","Joe Biden","Leo Messi","Cristiano Ronaldo","Shah Rukh Khan","Mark Zuckerberg","Narendra Modi","Sam Altman"]

In [10]:
dataframe=[]
def get_elon_musk_submissions_with_links(influencer ):
    subreddit_name='all'
     
    subreddit = reddit.subreddit(subreddit_name)

    # Search for submissions containing 'Elon Musk'
    submissions = subreddit.search(influencer, sort="relevance" ,time_filter="all",limit=None)

    # Print information about the submissions and their links
    for submission in submissions:
        dataframe.append([submission.title,influencer,submission.score,submission.num_comments,submission.url,submission.selftext,submission.permalink,submission.created_utc,submission.clicked])

    return dataframe
newls2d1=pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
infl =["Elon Musk", "Taylor Swift","Donald Trump","Joe Biden","Leo Messi","Cristiano Ronaldo","Shah Rukh Khan","Mark Zuckerberg","Narendra Modi","Sam Altman"]
for i in infl:
    fd=pd.DataFrame(get_elon_musk_submissions_with_links(influencer=i))
    newls2d1=pd.concat([newls2d1, fd], ignore_index=True)
newls2d1.rename(columns={0:'submission_title',1:'influencer',2:'submission_score',3:'submission_num_comments',4:'submission_URLs',5:'submission_selftext',6:'submission_permalink',7:'submission_created_utc',8:'submission_clicked'},inplace=True)
newls2d1['submission_created_date']=pd.to_datetime(newls2d1['submission_created_utc'],unit='s').dt.date
newls2d1.drop(['submission_created_utc'],axis=1,inplace=True)
newls2d1['new_submission_title']=newls2d1['submission_title'].apply(lambda x:finalpreprocess(x))
#newls2d1.drop_duplicates(inplace=True)
newls2d1.head(3)

,submission_title,influencer,submission_score,submission_num_comments,submission_URLs,submission_selftext,submission_permalink,submission_clicked,submission_created_date,new_submission_title
0,"Elon Musk, this you??",Elon Musk,29949,814,https://i.redd.it/xzghgi75xz3b1.jpg,,/r/WhitePeopleTwitter/comments/140br11/elon_musk_this_you/,False,2023-06-04,elon musk
1,Elon Musk is the damn CEO of racism at this point,Elon Musk,5110,1974,https://i.redd.it/smuxkgw1smbc1.jpeg,,/r/facepalm/comments/193asyz/elon_musk_is_the_damn_ceo_of_racism_at_this_point/,False,2024-01-10,elon musk damn ceo racism point
2,Elon Musk has gone full Nazi,Elon Musk,1601,105,https://i.redd.it/kbsrm8elkl0c1.jpeg,,/r/MarchAgainstNazis/comments/17w7kk2/elon_musk_has_gone_full_nazi/,False,2023-11-15,elon musk go full nazi


In [8]:
len(newls2d1)

2309

In [9]:
newls2d1.influencer.value_counts()

influencer
Sam Altman           244
Cristiano Ronaldo    243
Donald Trump         241
Taylor Swift         237
Leo Messi            232
Elon Musk            231
Joe Biden            231
Shah Rukh Khan       229
Mark Zuckerberg      221
Narendra Modi        200
Name: count, dtype: int64

In [4]:
dataframe=[]
def get_elon_musk_submissions_with_links(influencer ):
    subreddit_name='all'
    #num_results=500
    subreddit = reddit.subreddit(subreddit_name)

    # Search for submissions containing 'Elon Musk'
    submissions = subreddit.search(influencer, sort="relevance" ,time_filter="all",limit=None)

    # Print information about the submissions and their links
    for submission in submissions:
        dataframe.append([submission.title,influencer,submission.score,submission.num_comments,submission.url,submission.selftext,submission.permalink,submission.created_utc,submission.clicked])

    return dataframe
newls2d1=pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
infl =["Elon Musk", "Taylor Swift","Donald Trump","Joe Biden","Leo Messi","Cristiano Ronaldo","Shah Rukh Khan","Mark Zuckerberg","Narendra Modi","Sam Altman"]
for i in infl:
    fd=pd.DataFrame(get_elon_musk_submissions_with_links(influencer=i))
    newls2d1=pd.concat([newls2d1, fd], ignore_index=True)
newls2d1.rename(columns={0:'submission_title',1:'influencer',2:'submission_score',3:'submission_num_comments',4:'submission_URLs',5:'submission_selftext',6:'submission_permalink',7:'submission_created_utc',8:'submission_clicked'},inplace=True)
newls2d1['submission_created_date']=pd.to_datetime(newls2d1['submission_created_utc'],unit='s').dt.date
newls2d1.drop(['submission_created_utc'],axis=1,inplace=True)
newls2d1['new_submission_title']=newls2d1['submission_title'].apply(lambda x:finalpreprocess(x))
newls2d1.drop_duplicates(inplace=True)
newls2d1.head(3)

,submission_title,influencer,submission_score,submission_num_comments,submission_URLs,submission_selftext,submission_permalink,submission_clicked,submission_created_date,new_submission_title
0,"Elon Musk, this you??",Elon Musk,29943,814,https://i.redd.it/xzghgi75xz3b1.jpg,,/r/WhitePeopleTwitter/comments/140br11/elon_musk_this_you/,False,2023-06-04,elon musk
1,Elon Musk has gone full Nazi,Elon Musk,1595,105,https://i.redd.it/kbsrm8elkl0c1.jpeg,,/r/MarchAgainstNazis/comments/17w7kk2/elon_musk_has_gone_full_nazi/,False,2023-11-15,elon musk go full nazi
2,Elon Musk is the damn CEO of racism at this point,Elon Musk,5105,1976,https://i.redd.it/smuxkgw1smbc1.jpeg,,/r/facepalm/comments/193asyz/elon_musk_is_the_damn_ceo_of_racism_at_this_point/,False,2024-01-10,elon musk damn ceo racism point


In [7]:
newls2d1.influencer.value_counts()

influencer
Sam Altman           246
Taylor Swift         243
Donald Trump         240
Shah Rukh Khan       238
Leo Messi            236
Narendra Modi        229
Joe Biden            227
Elon Musk            226
Cristiano Ronaldo    220
Mark Zuckerberg      218
Name: count, dtype: int64

In [9]:
newls2d1.submission_created_date.sort_values(ascending=True)

4853     2013-11-19
4828     2013-12-11
4786     2014-03-14
12545    2014-09-30
3400     2014-10-03
            ...    
2228     2024-01-24
6528     2024-01-24
10262    2024-01-24
2131     2024-01-25
8277     2024-01-25
Name: submission_created_date, Length: 2323, dtype: object

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):

    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    #data_words = list(sent_to_words(data))# remove stop words
    #data_words = remove_stopwords(data_words)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return words_freq[:n]


import matplotlib.pyplot as plt
plt.figure(figsize = (16,9))
import seaborn as sns
most_common_bi = get_top_text_ngrams(newls2d1.submission_title,10,3)
most_common_bi = dict(most_common_bi)
sns.barplot(x=list(most_common_bi.values()),y=list(most_common_bi.keys()))

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
def create_wordcloud(model, topic):
    text = {word: value for word, value in model.show_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Topic" + " "+ str(topic))
    plt.show()


In [13]:
import gensim
from gensim.utils import simple_preprocess
import nltk
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pyLDAvis.gensim
warnings.simplefilter('always')
nltk.download('stopwords')
pyLDAvis.enable_notebook()
warnings.simplefilter('always')
import gensim.corpora as corpora#
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','elon','musk','donald','trump','elon musk','joe biden','taylor', 'swift','narendra modi'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]
def corpus(influencer):
  filterdf=newls2d1[newls2d1['influencer']==influencer]
  data = filterdf['submission_title'].values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)

  id2word = corpora.Dictionary(data_words)# Create Corpus
  texts = data_words# Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]
  return corpus
def id2word(influencer):
  filterdf=newls2d1[newls2d1['influencer']==influencer]
  data = filterdf['submission_title'].values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)

  id2word = corpora.Dictionary(data_words)# Create Corpus

  return data_words
def influencer(influencer):
  filterdf=newls2d1[newls2d1['influencer']==influencer]
  data = filterdf['submission_title'].values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)

  id2word = corpora.Dictionary(data_words)# Create Corpus
  texts = data_words# Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]# View
  num_topics = 10# Build LDA model
  lda_model = gensim.models.LdaModel(corpus,  id2word=id2word, num_topics=num_topics,passes=30, alpha='auto', eta='auto'
          )
  return lda_model




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mukundansankar1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:

  filterdf=newls2d1[newls2d1['influencer']=='Elon Musk']
  data = filterdf['submission_title'].values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)
  pd.Series(data_words)


0                                                                       []
1                                               [damn, ceo, racism, point]
2                                                       [gone, full, nazi]
3                                         [ceo, cto, rocket, company, ama]
4                                                                [opinion]
                                       ...                                
2245                                        [says, stepping, twitter, ceo]
2246                                                   [gets, owned, rule]
2247                   [called, sharing, private, photo, ex, amber, heard]
2248    [endorses, debunked, pizzagate, conspiracy, theory, deletes, post]
2249                                                     [nazi, confirmed]
Length: 2250, dtype: object

In [15]:
infl=newls2d1[newls2d1['influencer']==i]
  l=remove_stopwords(infl.submission_title.tolist())

  #l1=list(influencer('Elon Musk'))
  get_top_text_ngrams(l,100,3)

IndentationError: unexpected indent (2506568490.py, line 2)

In [16]:
from nltk.corpus import stopwords
stopwords=list(set(stopwords.words('english')))
stopwords.extend(['from','mark zuckerberg','shah','rukh','khan','cristiano ronaldo','donald trump', 'leo messi','taylor swift','subject', 're', 'edu', 'use','elon','musk','elon musk','donald','trump','elon musk','joe biden','taylor', 'swift','narendra modi'])

def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word.lower() not in stopwords:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array
newlst={}
for i in newls2d1['influencer'].unique():
  infl=newls2d1[newls2d1['influencer']==i]
  l=remove_stopwords(infl.submission_title.tolist())

  #l1=list(influencer('Elon Musk'))
  newlst[i]=get_top_text_ngrams(l,100,4)
 

In [17]:
newlst

{'Elon Musk': [('mark zuckerberg fight ain', 20),
  ('zuckerberg fight ain happening', 20),
  ('zambia according to checks', 20),
  ('according to checks notes', 20),
  ('to checks notes musk', 20),
  ('netflix resumes advertising controversy', 20),
  ('promises thermonuclear lawsuit brands', 20),
  ('thermonuclear lawsuit brands pause', 20),
  ('lawsuit brands pause ads', 20),
  ('says he stepping twitter', 20),
  ('he stepping twitter ceo', 20),
  ('loses 24 billion tesla', 20),
  ('24 billion tesla stock', 20),
  ('damn ceo racism point', 10),
  ('musk ceo cto rocket', 10),
  ('ceo cto rocket company', 10),
  ('cto rocket company ama', 10),
  ('cmv man child conman', 10),
  ('used illegal drugs worrying', 10),
  ('illegal drugs worrying leaders', 10),
  ('drugs worrying leaders tesla', 10),
  ('worrying leaders tesla spacex', 10),
  ('go beloved face internet', 10),
  ('beloved face internet despised', 10),
  ('face internet despised individual', 10),
  ('internet despised individua

In [18]:
import pickle

import pyLDAvis
import pyLDAvis.gensim_models
from pyLDAvis.gensim import prepare
from nltk.corpus import stopwords
pyLDAvis.enable_notebook()
stop_words=list(set(stopwords.words('english')))
stop_words.extend(['from', 'elon','musk','elon musk'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]
def func(i):
    filterdf=newls2d1[newls2d1['influencer']==i]
    data = filterdf['new_submission_title'].values.tolist()
    data_words = list(sent_to_words(remove_stopwords(data)))# remove stop words

    id2word = corpora.Dictionary(data_words)# Create Corpus
    texts = data_words# Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    num_topics = 10# Build LDA model
    lda_model = gensim.models.LdaModel(corpus,  id2word=id2word, num_topics=5,passes=15)

    vis = prepare(lda_model, corpus=corpus , dictionary=id2word)
    return vis
func('Elon Musk')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.203750  0.116804       1        1  24.317492
2     -0.014074 -0.134427       2        1  22.088012
1      0.246338  0.244204       3        1  20.347653
3     -0.188038  0.006056       4        1  19.626054
0      0.159524 -0.232636       5        1  13.620788, topic_info=           Term        Freq       Total Category  logprob  loglift
42      twitter  367.000000  367.000000  Default  30.0000  30.0000
61   zuckerberg   61.000000   61.000000  Default  29.0000  29.0000
60         mark   61.000000   61.000000  Default  28.0000  28.0000
129       fight   52.000000   52.000000  Default  27.0000  27.0000
6          nazi   39.000000   39.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
134        take    9.788113   19.787810   Topic5  -4.7789   1.2897
200          eu    9.787820   28.609122   Topic5  -4.7790   0.9210
291        must    9.787322   19.569070   Topic5  -4.7790   1.3007
19       spacex    9.786250   29.046539   Topic5  -4.7791   0.9057
110         car    9.785128   19.112467   Topic5  -4.7792   1.3241

[257 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
446       3  0.938490      accept
261       4  0.976119      accord
216       1  0.372754     account
216       4  0.613948     account
159       2  0.629194    actually
...     ...       ...         ...
108       1  0.410729       wrong
108       2  0.568702       wrong
260       2  0.901145   yesterday
263       4  0.976119      zambia
61        3  0.983099  zuckerberg

[278 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 4, 1])

In [19]:
from nltk.corpus import stopwords
stop_words=list(set(stopwords.words('english')))
stop_words.extend(['from','taylor','swift','taylor swift','subject', 're'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]

func('Taylor Swift')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.111611  0.279365       1        1  21.338999
3      0.111399 -0.018516       2        1  21.174091
4      0.085173 -0.071902       3        1  20.848598
2      0.046009 -0.224679       4        1  20.056520
0     -0.354192  0.035732       5        1  16.581792, topic_info=         Term        Freq       Total Category  logprob  loglift
35    version  103.000000  103.000000  Default  30.0000  30.0000
82       year   96.000000   96.000000  Default  29.0000  29.0000
1         fan   97.000000   97.000000  Default  28.0000  28.0000
81       time   54.000000   54.000000  Default  27.0000  27.0000
80     person   53.000000   53.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
248       saw   10.667861   20.592018   Topic5  -4.8542   1.1392
443     super   10.664374   20.591597   Topic5  -4.8546   1.1389
353  together   10.663206   20.682542   Topic5  -4.8547   1.1344
326     leave   10.662554   20.437947   Topic5  -4.8547   1.1462
280     think   10.660641   20.370815   Topic5  -4.8549   1.1493

[272 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
426       4  0.907915  action
4         1  0.371425      ai
4         2  0.116070      ai
4         3  0.208927      ai
4         4  0.301783      ai
...     ...       ...     ...
153       4  0.951185   wilde
237       1  0.663775    work
237       3  0.314420    work
82        3  0.186532    year
82        5  0.808304    year

[302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 3, 1])

In [20]:
from nltk.corpus import stopwords
stop_words=list(set(stopwords.words('english')))
stop_words.extend(['from','donald trump','subject', 're', 'edu', 'use','donald','trump'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]
func('Donald Trump')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.150883  0.075055       1        1  23.680068
0     -0.200031  0.243279       2        1  21.860639
1      0.222768  0.019254       3        1  20.421923
3     -0.168330 -0.191756       4        1  18.289119
4     -0.005290 -0.145832       5        1  15.748252, topic_info=          Term       Freq      Total Category  logprob  loglift
50        fuck  52.000000  52.000000  Default  30.0000  30.0000
97     carroll  39.000000  39.000000  Default  29.0000  29.0000
101       jean  39.000000  39.000000  Default  28.0000  28.0000
7         lose  37.000000  37.000000  Default  27.0000  27.0000
1    president  82.000000  82.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
204     speech   7.922169  23.429746   Topic5  -5.2080   0.7641
433       like   7.921608  23.434146   Topic5  -5.2081   0.7638
201      gaffe   7.921359  23.434176   Topic5  -5.2081   0.7638
385   threaten   7.919968  16.067360   Topic5  -5.2083   1.1410
8         mind   7.919482  23.383275   Topic5  -5.2083   0.7657

[290 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
575       2  0.958023  accurate
163       2  0.964500    accuse
125       4  0.638453        ad
125       5  0.340508        ad
408       1  0.951271     admit
...     ...       ...       ...
58        5  0.499627     would
290       1  0.646797      year
290       4  0.344959      year
180       1  0.661684      york
180       3  0.330842      york

[362 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4, 5])

In [21]:
from nltk.corpus import stopwords
stop_words=list(set(stopwords.words('english')))
stop_words.extend(['from','subject', 're', 'edu', 'use','narendra', 'modi','narendra modi'])
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]
func('Narendra Modi')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.072778  0.101913       1        1  27.781041
4      0.103085 -0.159591       2        1  24.357001
3      0.001323  0.120169       3        1  18.826322
0      0.028685 -0.017010       4        1  17.807191
1     -0.205872 -0.045482       5        1  11.228445, topic_info=         Term       Freq      Total Category  logprob  loglift
26    stadium  57.000000  57.000000  Default  30.0000  30.0000
4         jai   8.000000   8.000000  Default  29.0000  29.0000
117     world  21.000000  21.000000  Default  28.0000  28.0000
76     indian  37.000000  37.000000  Default  27.0000  27.0000
253  congress  10.000000  10.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
98      modis   3.068149  14.958371   Topic5  -4.7210   0.6025
22       year   3.070452  14.995262   Topic5  -4.7202   0.6008
25         pm   3.029499  94.529640   Topic5  -4.7337  -1.2538
99       talk   1.611278   8.678780   Topic5  -5.3650   0.5029
200    indias   1.611024  11.865530   Topic5  -5.3652   0.1900

[290 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
38        2  0.252289     accent
38        4  0.756867     accent
455       3  0.838790     accord
241       3  0.753818     across
643       5  0.890135     actual
...     ...       ...        ...
22        3  0.133375       year
22        4  0.333439       year
22        5  0.200063       year
696       4  0.843825  yesterday
560       4  0.843825  yugpurush

[353 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 2])